In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os
from pathlib import Path

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
def plot_series(series, y=None, y_pred=None, x_label="$t$", y_label="$x(t)$"):
    plt.plot(series, ".-")
    if y is not None:
        plt.plot(n_steps, y, "bx", markersize=10)
    if y_pred is not None:
        plt.plot(n_steps, y_pred, "ro")
    plt.grid(True)
    if x_label:
        plt.xlabel(x_label, fontsize=16)
    if y_label:
        plt.ylabel(y_label, fontsize=16, rotation=0)
    plt.hlines(0, 0, 100, linewidth=1)
    plt.axis([0, n_steps + 1, -1, 1])


# 15.3 시계열 예측하기

In [ ]:
#단변량 시계열 생성

def generate_time_series(batch_size,n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4,batch_size,1)
  time = np.linspace(0,1,n_steps) # 0...n_steps..1
  series = 0.5 * np.sin((time-offsets1) * (freq1*10+10)) #   wave 1
  series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + noise
  return series[..., np.newaxis].astype(np.float32)



In [ ]:
n_steps= 50
series = generate_time_series(10000,n_steps+1)
print(series.shape)
#(10000,51,1) 배치크기, 타임스탭 수, 차원 수(단변량)

In [ ]:
X_train, y_train = series[:7000, :n_steps], series[:7000,-1]
X_valid,y_valid = series[7000:9000, :n_steps], series[7000:9000,-1]
X_test,y_test = series[9000:,n_steps],series[9000:,-1]

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(12, 4))
for col in range(3):
    plt.sca(axes[col])
    plot_series(X_valid[col, :, 0], y_valid[col, 0],
                y_label=("$x(t)$" if col==0 else None))
save_fig("time_series_plot")
plt.show()

### 15.3.1 기준 성능

1.순진한 예측

마지막값 그대로 사용



In [ ]:
y_pred = X_valid[:, -1]
np.mean(keras.losses.mean_squared_error(y_valid, y_pred))

2.선형 예측 

시계열 값의 선형조합으로 예측. 각 타임스텝이 변수. 
50개 변수의 선형모델

In [ ]:
model= keras.models.Sequential([
                                keras.layers.Flatten(input_shape=[50,1]),
                                keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(X_train,y_train,epochs=20,
                    validation_data = (X_valid,y_valid))

In [ ]:
model.evaluate(X_valid,y_valid)

### 15.3.2 간단한 RNN 구현하기

In [ ]:
#하나의 뉴런, 하나의 층.

#어떤 길이도 처리할 수 있기 때문에 길이지정 안함 == None.
model =keras.models.Sequential([                                
                                keras.layers.SimpleRNN(1,input_shape=[None,1])
])

### 15.3.3 심층 RNN

In [ ]:
#셀을 여러층.

model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20,return_sequences=True),
                                 keras.layers.SimpleRNN(1)
])

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20),
                                 keras.layers.Dense(1)
])

### 15.3.4 여러 타임 스텝 앞을 예측하기

1. 계속 이어서 예측

In [ ]:
series = generate_time_series(1,n_steps+10)

X_new,y_new =series[:,:n_steps], series[:,n_steps]

In [ ]:
X=X_new
X.shape

In [ ]:
for step_ahead in range(10):
  #계속 50개씩 보고 예측.
  y_pred_one = model.predict(X[:, step_ahead:])[:,np.newaxis,:] #(1,1,1)로.
  X=np.concatenate([X,y_pred_one],axis=1)#이어붙이기
  print(X.shape)

Y_pred = X[:,n_steps]

In [ ]:
Y_pred.shape

2. 한번에 전부 예측

seq to vec을 이용하는데. 값 10개를 출력. (타겟을 10개짜리로 바꿔야함) 

In [ ]:
series = generate_time_series(10000, n_steps+10)
X_train,y_train = series[:7000,:n_steps], series[:7000,-10:,0]
X_valid,y_valid = series[7000:9000,:n_steps],series[7000:9000,-10:,0]
X_test,y_test = series[9000:,:n_steps],series[9000:,-10:,0]

In [ ]:
model = keras.models.Sequential([
                                keras.layers.SimpleRNN(20,return_sequences=True,input_shape=[None,1]),
                                keras.layers.SimpleRNN(20),
                                keras.layers.Dense(10)
])

In [ ]:
model.compile(loss="mse",optimizer="adam")
history = model.fit(X_train,y_train, epochs=20,
                    validation_data = (X_valid,y_valid))

3. seq to seq로 모든 타임스탭에서 다음10개를 예측

In [ ]:
Y=np.empty((10000,n_steps,10))

for step_ahead in range(1,10+1):
  #브로드캐스팅 이용. 
#조건1. 차원에서 요소수가 동등하다. 
#조건2. 둘중 하나 차원의 요소수가 1이다
  Y[:,:,step_ahead-1] = series[:,step_ahead:step_ahead+n_steps,0]
#y의 0부터 9까지 10개 채우는 과정
#배치의 각 타임스탭의 0번째~9번째 성분을 각각 step_ahead만큼 앞 것으로 지정해줌.

Y_train=Y[:7000]
Y_valid=Y[7000:9000]
Y_test = Y[9000:]

In [ ]:
series.shape

In [ ]:
Y.shape

In [ ]:
series[:,1:1+n_steps,0].shape #1~50번쨰를 0~49번쨰에 매칭 --> 하나 앞

In [ ]:
series[:,2:2+n_steps,0].shape #2~51번쨰를 0~49번쨰에 매칭 --> 두개 앞 ... 이런식.
# 차원계산이 은근 어렵네..


In [ ]:
model = keras.models.Sequential([
                                keras.layers.SimpleRNN(20,return_sequences=True,
                                                        input_shape=[None,1]),
                                 keras.layers.SimpleRNN(20,return_sequences=True),
                                 keras.layers.TimeDistributed(keras.layers.Dense(10))
                                 #모든층에 dense를 적용해야함.
                                 #TimeDistributed층은 다른층(ex dense)을 감싸서 입력 시퀀스의 모든 타임스텝에 적용해줌.
                                 #각 타임스탭을 별개의 샘플처럼 다루도록 해줌.
])

훈련은 모든 출력 이용

예측과 평가는 마지막 타임 스텝만 이용

In [ ]:
def last_time_step_mse(Y_true, Y_pred):
  return keras.metrics.mean_squared_error(Y_true[:,-1],Y_pred[:,-1])

In [ ]:
optimizer = keras.optimizers.Adam(lr=0.01)
model.compile(loss="mse", optimizer = optimizer, metrics =[last_time_step_mse])

## 15.4 긴 시퀀스 다루기

### 15.4.1 불안정한 그레디언트 문제와 싸우기

심층 신경망과 같이

좋은 초기화, 빠른 옵티마이저, 드롭아웃, BN같은 방법이 쓰일 수 있음.

BN은 은닉층이 아닌 입력에 적용했을때만 효과적이라고 밝혀짐.

층 정규화(layer normalization) 도 사용됨.
층 정규화는 배치단위가 아닌 특성단위로 정규화함.

In [ ]:
X_train.shape, Y_train.shape

In [ ]:
#층 정규화 구현

class LNSimpleRNNCell(keras.layers.Layer):
  def __init__(self,units,activation="tanh",**kwargs):
    super().__init__(**kwargs)
    self.state_size=units
    self.output_size=units
    self.simple_rnn_cell = keras.layers.SimpleRNNCell(units,
                                                      activation=None)
    self.layer_norm = keras.layers.LayerNormalization()
    self.activation = keras.activations.get(activation)







  def call(self,inputs,states):
    outputs, new_states = self.simple_rnn_cell(inputs,states)
    norm_outputs = self.activation(self.layer_norm(outputs))
    return norm_outputs, [norm_outputs]

In [ ]:
model= keras.models.Sequential([
                                keras.layers.RNN(LNSimpleRNNCell(20),
                                                 return_sequences=True,
                                                 input_shape=[None,1]),
                                keras.layers.RNN(LNSimpleRNNCell(20),
                                                 return_sequences=True),
                                keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
X_valid.shape

In [ ]:
Y_valid.shape

In [ ]:
model.compile(loss="mse",optimizer="adam",
              metrics=[last_time_step_mse])

history = model.fit(X_train,Y_train,epochs=20,
                    validation_data = (X_valid,Y_valid))

In [ ]:
model.evaluate(X_test,Y_test)

### 15.4.2 단기 기억 문제 해결하기

LSTM 셀

방법1. LSTM셀 직접추가 (GPU에서 빨라서 선호)

In [ ]:
model =keras.models.Sequential([
                                keras.layers.LSTM(20,
                                                  return_sequences=True,
                                                  input_shape=[None,1]),
                                keras.layers.LSTM(20,
                                                  return_sequences=True),
                                keras.layers.TimeDistributed(keras.layers.Dense(10))
])

방법2. keras.layers.RNN 층에 매개변수로 추가. (사용자 정의 셀 구현시 사용)

In [ ]:
model=keras.models.Sequential([
                               
                               keras.layers.RNN(keras.layers.LSTMCell(20),
                                                return_sequences=True,
                                                input_shape=[None,1]),
                               keras.layers.RNN(keras.layers.LSTMCell(20),
                                                return_sequences=True),
                               keras.layers.TimeDistributed(keras.layers.Dense(10))
                               ])


GRU 셀

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

model = keras.models.Sequential([
                                 
                                 keras.layers.GRU(20,return_sequences=True,
                                                  input_shape=[None,1]),
                                 keras.layers.GRU(20,return_sequences=True),
                                 keras.layers.TimeDistributed(keras.layers.Dense(10))
])

1D 합성곱 층을 사용해 시퀀스 처리하기

In [ ]:
#1d 합성곱을 통해 시퀀스 길이를 줄여주면 좀 더 긴 패턴을 학습하기 좋음.
model = keras.models.Sequential([
                                keras.layers.Conv1D(filters=20,kernel_size=4,
                                                    strides=2,padding="valid",
                                                    input_shape=[None,1]),
                                keras.layers.GRU(20,return_sequences=True),
                                keras.layers.GRU(20,return_sequences=True),
                                keras.layers.TimeDistributed(keras.layers.Dense(10))                     
])

In [ ]:
model.compile(loss="mse",
              optimizer="adam",
              metrics=[last_time_step_mse])
history = model.fit(X_train,Y_train[:,3::2],epochs=20,
                    validation_data = (X_valid,Y_valid[:,3::2]))

WAVE NET

뉴런간 간격(팽창비율)을 늘리면서 학습.

2칸 - 4칸 -8칸 -.....

하위층에서는 단기패턴 학습

상위층에서는 장기패턴 학습

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None,1]))

for rate in (1,2,4,8)*2:
  model.add(keras.layers.Conv1D(filters=20,
                                kernel_size=2,
                                padding="causal",
                                activation="relu",
                                dilation_rate=rate))
  
model.add(keras.layers.Conv1D(filters=10,kernel_size=1))
model.compile(loss="mse",optimizer="adam",metrics=[last_time_step_mse])
history= model.fit(X_train,Y_train,epochs=20,
                   validation_data=(X_valid,Y_valid))

# 연습문제9.sketchRNN dataset 으로 분류모델 만들기

데이터셋 불러오기(tfds 아님. tfrecords)

In [ ]:
DOWNLOAD_ROOT = "http://download.tensorflow.org/data/"
FILENAME = "quickdraw_tutorial_dataset_v1.tar.gz"

filepath = keras.utils.get_file(FILENAME,
                                DOWNLOAD_ROOT + FILENAME,
                                cache_subdir = "datasets/quickdraw",
                                extract=True)

In [ ]:
print(filepath)
quickdraw_dir = Path(filepath).parent
print(quickdraw_dir)

In [ ]:
from os import listdir
sorted(listdir(quickdraw_dir))

In [ ]:
train_files = sorted([str(path) for path in quickdraw_dir.glob("training.tfrecord-*")])
eval_files = sorted([str(path) for path in quickdraw_dir.glob("eval.tfrecord-*")])

트레인 tfrecords 파일들

In [ ]:
train_files

평가 트레인 tfrecords 파일들

In [ ]:
eval_files

class 확인

In [ ]:
with open(quickdraw_dir/"eval.tfrecord.classes") as test_classes_file:
  test_classes = test_classes_file.readlines()

with open(quickdraw_dir/"training.tfrecord.classes") as train_classes_file:
  train_classes = train_classes_file.readlines()

In [ ]:
assert train_classes ==test_classes

In [ ]:
class_names = [name.strip().lower() for name in train_classes]

In [ ]:
sorted(class_names)

example 프로토콜 버퍼 파싱하기

In [ ]:
def parse(data_batch):
  #파싱을 위한 설명
  feature_descriptions={
      "ink": tf.io.VarLenFeature(dtype=tf.float32),
      "shape": tf.io.FixedLenFeature([2],dtype=tf.int64),
      "class_index": tf.io.FixedLenFeature([1], dtype=tf.int64)
  }
  #배치 단위로 파싱
  examples = tf.io.parse_example(data_batch, feature_descriptions)
  #변동길이 특성은 희소텐서로 파싱됨. --> 밀집으로 변경
  flat_sketches = tf.sparse.to_dense(examples["ink"])
  #batchsize,변동길이,컬러 로 reshape
  sketches = tf.reshape(flat_sketches, shape = [tf.size(data_batch),-1,3])
  lengths = examples["shape"][:,0]
  labels = examples["class_index"][:,0]

  return sketches, lengths, labels

In [ ]:
def quickdraw_dataset(filepaths, batch_size=32,shuffle_buffer_size=None,
                      n_parse_threads =5, n_read_threads= 5,
                      cache=False):
  #tfrecords불러오기
  dataset = tf.data.TFRecordDataset(filepaths,
                                    num_parallel_reads = n_read_threads)
  if cache:
    dataset = dataset.cache()
  if shuffle_buffer_size :
    dataset = dataset.shuffle(shuffle_buffer_size)
  dataset = dataset.batch(batch_size)
  #텐서로 파싱하기
  dataset = dataset.map(parse, num_parallel_calls = n_parse_threads)
  return dataset.prefetch(1)


In [ ]:
train_set = quickdraw_dataset(train_files, shuffle_buffer_size=10000)
valid_set = quickdraw_dataset(eval_files[:5])
test_set = quickdraw_dataset(eval_files[5:])

In [ ]:
for sketches,lengths, labels in train_set.take(1):
  print("sketchs = ",sketches)
  print("lengths = ",lengths)
  print("labels = ",labels)

시각화(이해만 하고 넘어감)

In [ ]:
def draw_sketch(sketch, label=None):
    origin = np.array([[0., 0., 0.]])
    sketch = np.r_[origin, sketch]
    #획이 끝나는 인덱스들
    stroke_end_indices = np.argwhere(sketch[:, -1]==1.)[:, 0]
    #좌표
    coordinates = np.cumsum(sketch[:, :2], axis=0)
    #획을 나누는것 같음.
    strokes = np.split(coordinates, stroke_end_indices + 1)
    title = class_names[label.numpy()] if label is not None else "Try to guess"
    plt.title(title)
    plt.plot(coordinates[:, 0], -coordinates[:, 1], "y:")
    for stroke in strokes:
        plt.plot(stroke[:, 0], -stroke[:, 1], ".-")
    plt.axis("off")

def draw_sketches(sketches, lengths, labels):
    n_sketches = len(sketches)
    n_cols = 4
    n_rows = (n_sketches - 1) // n_cols + 1
    plt.figure(figsize=(n_cols * 3, n_rows * 3.5))
    for index, sketch, length, label in zip(range(n_sketches), sketches, lengths, labels):
        plt.subplot(n_rows, n_cols, index + 1)
        draw_sketch(sketch[:length], label)
    plt.show()

for sketches, lengths, labels in train_set.take(1):
    draw_sketches(sketches, lengths, labels)

In [ ]:
lengths = np.concatenate([lengths for _,lengths,_ in train_set.take(1000)])
print(max(lengths))
print(min(lengths))
plt.hist(lengths, bins=150, density=True)
plt.axis([0,200,0,0.03])
plt.xlabel("length")
plt.ylabel("density")
plt.show()

너무 긴 획 자르기

In [ ]:
def crop_long_sketches(dataset, max_length=100):
  return dataset.map(lambda inks, lengths, labels : (inks[:, :max_length], labels))

In [ ]:
cropped_train_set = crop_long_sketches(train_set)
cropped_valid_set = crop_long_sketches(valid_set)
cropped_test_set = crop_long_sketches(test_set)

In [ ]:
model = keras.models.Sequential([
                                 keras.layers.Conv1D(32,5,strides=2,activation="relu"),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Conv1D(64,5,strides=2,activation="relu"),
                                 keras.layers.BatchNormalization(),
                                 keras.layers.Conv1D(128,3,strides=2,activation="relu"),
                                 keras.layers.BatchNormalization(),

                                 keras.layers.LSTM(128,return_sequences=True),
                                 keras.layers.LSTM(128),
                                 keras.layers.Dense(len(class_names),activation="softmax")
])

In [ ]:
optimizer =keras.optimizers.SGD(lr=1e-2, clipnorm=1.)
model.compile(loss=keras.losses.sparse_categorical_crossentropy,
              optimizer= optimizer,
              metrics = ["accuracy","sparse_top_k_categorical_accuracy"])

history = model.fit(cropped_train_set,epochs=2,
                    validation_data = cropped_valid_set)

In [ ]:
y_test = np.concatenate([labels for _,_,labels in test_set])
y_probas =model.predict(test_set)

In [ ]:
np.mean(keras.metrics.sparse_top_k_categorical_accuracy(y_test,y_probas))
#sparse_top_k_categorical_accuracy(y_true, y_pred, k=5)
#확률받아서 정확도 측정.
#ex)
#y_true = [2, 1]
#y_pred = [[0.1, 0.9, 0.8], [0.05, 0.95, 0]]

In [ ]:
n_new =10
Y_probas = model.predict(sketches)
top_k = tf.nn.top_k(Y_probas,k=5)

for index in range(n_new):
  plt.figure(figsize=(3,3.5))
  draw_sketch(sketches[index])
  plt.show()
  print("Top-5 predictions: ".format(index+1))
  for k in range(5):
    class_name = class_names[top_k.indices[index,k]]
    proba = 100 * top_k.values[index,k]
    print(" {}. {} {:.3f}%".format(k+1, class_name, proba))
  print("Answer: {}".format(class_names[labels[index]]))

In [ ]:
model.save("my_sketchrnn")

In [ ]:
model.summary()

# 연습문제10.바흐음악

In [ ]:
DOWNLOAD_ROOT = "https://github.com/ageron/handson-ml2/raw/master/datasets/jsb_chorales/"
FILENAME = "jsb_chorales.tgz"
filepath =keras.utils.get_file(FILENAME,
                               DOWNLOAD_ROOT+FILENAME,
                               cache_subdir = "datasets/jsb_chorales",
                               extract=True)

In [ ]:
jsb_chorales_dir =Path(filepath).parent
listdir(jsb_chorales_dir)

In [ ]:
train_files = sorted(jsb_chorales_dir.glob("train/chorale_*.csv"))
valid_files = sorted(jsb_chorales_dir.glob("valid/chorale_*.csv"))
test_files = sorted(jsb_chorales_dir.glob("test/chorale_*.csv"))

In [ ]:
import pandas as pd

pd.read_csv(train_files[0]).head()

In [ ]:
def load_chorales(filepaths):
  return [pd.read_csv(filepath).values.tolist() for filepath in filepaths]

In [ ]:
train_chorales = load_chorales(train_files)
valid_chorales = load_chorales(valid_files)
test_chorales = load_chorales(test_files)

In [ ]:
notes = set()
for chorales in (train_chorales, valid_chorales, test_chorales):
    for chorale in chorales:
        for chord in chorale:
            notes |= set(chord)

n_notes = len(notes)
min_note = min(notes - {0})
max_note = max(notes)

assert min_note == 36
assert max_note == 81

In [ ]:
notes

연주를 위한 함수. 이해안해도됨.

In [ ]:
from IPython.display import Audio

def notes_to_frequencies(notes):
    # Frequency doubles when you go up one octave; there are 12 semi-tones
    # per octave; Note A on octave 4 is 440 Hz, and it is note number 69.
    return 2 ** ((np.array(notes) - 69) / 12) * 440

def frequencies_to_samples(frequencies, tempo, sample_rate):
    note_duration = 60 / tempo # the tempo is measured in beats per minutes
    # To reduce click sound at every beat, we round the frequencies to try to
    # get the samples close to zero at the end of each note.
    frequencies = np.round(note_duration * frequencies) / note_duration
    n_samples = int(note_duration * sample_rate)
    time = np.linspace(0, note_duration, n_samples)
    sine_waves = np.sin(2 * np.pi * frequencies.reshape(-1, 1) * time)
    # Removing all notes with frequencies ≤ 9 Hz (includes note 0 = silence)
    sine_waves *= (frequencies > 9.).reshape(-1, 1)
    return sine_waves.reshape(-1)

def chords_to_samples(chords, tempo, sample_rate):
    freqs = notes_to_frequencies(chords)
    freqs = np.r_[freqs, freqs[-1:]] # make last note a bit longer
    merged = np.mean([frequencies_to_samples(melody, tempo, sample_rate)
                     for melody in freqs.T], axis=0)
    n_fade_out_samples = sample_rate * 60 // tempo # fade out last note
    fade_out = np.linspace(1., 0., n_fade_out_samples)**2
    merged[-n_fade_out_samples:] *= fade_out
    return merged

def play_chords(chords, tempo=160, amplitude=0.1, sample_rate=44100, filepath=None):
    samples = amplitude * chords_to_samples(chords, tempo, sample_rate)
    if filepath:
        from scipy.io import wavfile
        samples = (2**15 * samples).astype(np.int16)
        wavfile.write(filepath, sample_rate, samples)
        return display(Audio(filepath))
    else:
        return display(Audio(samples, rate=sample_rate))

In [ ]:
for index in range(3):
    play_chords(train_chorales[index])

새로운 곡을 만들기위해 우리는 다음 4개의 코드를 예측하는 모델을 훈련하고싶다.

방법1. 4개의 노트를 한번에 예측할 수있다.

방법2. 한번에 하나의 노트를 예측할수 있다.

방법2로 seq to seq 구조를 이용. 바로 다음 노트를 예측해보자.

노트 36 to 81을 0 to 45로 변경

128윈도우로 모델을 학습. 128윈도우는 32코드

Tf data를 이용해 전처리

In [ ]:
def create_target(batch):
    X = batch[:, :-1]
    Y = batch[:, 1:] #두번쨰부터 마지막까지가 타켓이됨(다음노트들)
    return X, Y

def preprocess(window):
    window = tf.where(window == 0, window, window - min_note + 1) 
    #0은 그대로 0으로, 나머지는 0~45로 min_note만큼 shift
    return tf.reshape(window, [-1]) # convert to arpegio

def bach_dataset(chorales, batch_size=32, shuffle_buffer_size=None,
                 window_size=32, window_shift=16, cache=True):
    def batch_window(window):
        return window.batch(window_size + 1)

    def to_windows(chorale):
        dattaset = tf.daa.Dataset.from_tensor_slices(chorale)
        
        #shift는 윈도우가 몇씩 이동하는지. 
        #ex winodw size=3. shift =1 
        #1,2,3
        #2,3,4
        #3,4,5
        dataset = dataset.window(window_size + 1, window_shift, drop_remainder=True)
        return dataset.flat_map(batch_window)
        #flat_map은 평탄화.
        #   [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
        #-->[1,2,3,4,5,6,7,8,9]

    chorales = tf.ragged.constant(chorales, ragged_rank=1)
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
    dataset = dataset.flat_map(to_windows).map(preprocess)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(create_target)
    return dataset.prefetch(1)

In [ ]:
train_set = bach_dataset(train_chorales, shuffle_buffer_size=1000)
valid_set = bach_dataset(valid_chorales)
test_set = bach_dataset(test_chorales)

In [ ]:
n_embedding_dims = 5

model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=n_notes, output_dim=n_embedding_dims,
                           input_shape=[None]),
    keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
    keras.layers.BatchNormalization(),
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.Dense(n_notes, activation="softmax")
])

model.summary()

In [ ]:
ptimizer = keras.optimizers.Nadam(lr=1e-3)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
model.fit(train_set, epochs=20, validation_data=valid_set)

In [ ]:
model.save("my_bach_model.h5")
model.evaluate(test_set)

In [ ]:
def generate_chorale(model, seed_chords, length):
    arpegio = preprocess(tf.constant(seed_chords, dtype=tf.int64))
    arpegio = tf.reshape(arpegio, [1, -1])
    for chord in range(length):
        for note in range(4):
            #next_note = model.predict_classes(arpegio)[:1, -1:]
            next_note = np.argmax(model.predict(arpegio), axis=-1)[:1, -1:]
            arpegio = tf.concat([arpegio, next_note], axis=1)
    arpegio = tf.where(arpegio == 0, arpegio, arpegio + min_note - 1)
    return tf.reshape(arpegio, shape=[-1, 4])

In [ ]:
seed_chords = test_chorales[2][:8]
play_chords(seed_chords, amplitude=0.2)


In [ ]:
new_chorale = generate_chorale(model, seed_chords, 56)
play_chords(new_chorale)

In [ ]:
def generate_chorale_v2(model, seed_chords, length, temperature=1):
    arpegio = preprocess(tf.constant(seed_chords, dtype=tf.int64))
    arpegio = tf.reshape(arpegio, [1, -1])
    for chord in range(length):
        for note in range(4):
            next_note_probas = model.predict(arpegio)[0, -1:]
            rescaled_logits = tf.math.log(next_note_probas) / temperature
            next_note = tf.random.categorical(rescaled_logits, num_samples=1)
            arpegio = tf.concat([arpegio, next_note], axis=1)
    arpegio = tf.where(arpegio == 0, arpegio, arpegio + min_note - 1)
    return tf.reshape(arpegio, shape=[-1, 4])

In [ ]:
new_chorale_v2_cold = generate_chorale_v2(model, seed_chords, 56, temperature=0.8)
play_chords(new_chorale_v2_cold, filepath="bach_cold.wav")

In [ ]:
new_chorale_v2_medium = generate_chorale_v2(model, seed_chords, 56, temperature=1.0)
play_chords(new_chorale_v2_medium, filepath="bach_medium.wav")

In [ ]:
new_chorale_v2_hot = generate_chorale_v2(model, seed_chords, 56, temperature=1.5)
play_chords(new_chorale_v2_hot, filepath="bach_hot.wav")


In [ ]:
play_chords(test_chorales[2][:64], filepath="bach_test_4.wav")